In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import nltk
import numpy as np

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import pandas as pd
pd.set_option('display.max_colwidth',200)
df=pd.read_csv(r'/content/drive/MyDrive/1st project googlecolab/GrammarandProductReviews.csv',encoding='UTF-8',sep=',')
df.shape
df.head()

,reviews,rating
0,!We enjoy watching this movie with our kids! It's enjoyable for the whole family,5
1,""" Not best sequel but one of better ones"" it builds on ID4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",3
2,"""Best pressed powder I've ever used. It is so smooth going on my face, and it looks natural.",5
3,"""Dark Shadows"" is Tim Burton's take on the TV show of the same name and a really cool one, at that. Great cast and a cameo of Alice Cooper. Doesn't get much better.Johnny Depp continues a successf...",4
4,"""Easy to assemble...lightweight....yet seems sturdyHome to sewing supplies, portable machine is on BOTTOM shelf, just in case!So far, so goodBegan with two Way basics products, now own five......L...",5


In [4]:
df.isna().sum()

reviews    49
rating      0
dtype: int64

In [5]:
df.dropna(subset = ["reviews"], inplace=True)

In [6]:
df.reviews = df.reviews.astype(str) 
df.rating = df.rating.astype(int) 
df.reviews = df.reviews.apply(lambda x: x.lower())
df.head()

,reviews,rating
0,!we enjoy watching this movie with our kids! it's enjoyable for the whole family,5
1,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",3
2,"""best pressed powder i've ever used. it is so smooth going on my face, and it looks natural.",5
3,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. doesn't get much better.johnny depp continues a successf...",4
4,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",5


In [7]:
import sys  
!{sys.executable} -m pip install contractions

In [8]:
import contractions
def contract(text):
  txt = contractions.fix(text)
  return txt

df['decon'] = df['reviews'].apply(lambda com : contract(com))

In [9]:
df.head()

,reviews,rating,decon
0,!we enjoy watching this movie with our kids! it's enjoyable for the whole family,5,!we enjoy watching this movie with our kids! it is enjoyable for the whole family
1,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",3,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ..."
2,"""best pressed powder i've ever used. it is so smooth going on my face, and it looks natural.",5,"""best pressed powder I have ever used. it is so smooth going on my face, and it looks natural."
3,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. doesn't get much better.johnny depp continues a successf...",4,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. does not get much better.johnny depp continues a success..."
4,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",5,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l..."


In [10]:
def decontracted(phrase):
    # phrase = re.sub(r"\'t", "not", phrase)
    # phrase = re.sub(r"\'re", " are", phrase)
    # phrase = re.sub(r"\'s", " is", phrase)
    # phrase = re.sub(r"\'d", " would", phrase)
    # phrase = re.sub(r"\'ll", " will", phrase)
    # phrase = re.sub(r"\'ve", " have", phrase)
    # phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub('\W', ' ', phrase)
    phrase = re.sub('\s+', ' ', phrase)
    phrase = phrase.strip(' ')
    return phrase
df['decons'] = df['decon'].apply(lambda com : decontracted(com))


In [11]:
import string
def remove_punctuations(txt):
    txt = "".join([c for c in txt if c not in string.punctuation])
    return txt
df['RemovePunct'] = df['decons'].apply(remove_punctuations)
df.head()

,reviews,rating,decon,decons,RemovePunct
0,!we enjoy watching this movie with our kids! it's enjoyable for the whole family,5,!we enjoy watching this movie with our kids! it is enjoyable for the whole family,we enjoy watching this movie with our kids it is enjoyable for the whole family,we enjoy watching this movie with our kids it is enjoyable for the whole family
1,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",3,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",not best sequel but one of better ones it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any...,not best sequel but one of better ones it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any...
2,"""best pressed powder i've ever used. it is so smooth going on my face, and it looks natural.",5,"""best pressed powder I have ever used. it is so smooth going on my face, and it looks natural.",best pressed powder I have ever used it is so smooth going on my face and it looks natural,best pressed powder I have ever used it is so smooth going on my face and it looks natural
3,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. doesn't get much better.johnny depp continues a successf...",4,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. does not get much better.johnny depp continues a success...",dark shadows is tim burton s take on the tv show of the same name and a really cool one at that great cast and a cameo of alice cooper does not get much better johnny depp continues a successful p...,dark shadows is tim burton s take on the tv show of the same name and a really cool one at that great cast and a cameo of alice cooper does not get much better johnny depp continues a successful p...
4,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",5,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",easy to assemble lightweight yet seems sturdyhome to sewing supplies portable machine is on bottom shelf just in case so far so goodbegan with two way basics products now own five love the clean l...,easy to assemble lightweight yet seems sturdyhome to sewing supplies portable machine is on bottom shelf just in case so far so goodbegan with two way basics products now own five love the clean l...


In [12]:

#> the cat can be sit with the bat on the striped mat under many fly geese

In [13]:
# import nltk

# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
# lemmatizer = nltk.stem.WordNetLemmatizer()
# def lemmatize_text(text):
#   numList = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
#   separator = ' '
#   a=separator.join(numList)
#   return a

# df["text_lemmatizednew"] = df['RemovePunct'].apply(lemmatize_text)
# df.head()

In [14]:
stop =['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours','by','hers', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',  'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',  'couldn',  'didn',  'doesn',  'hadn',  'hasn',  'haven',  'isn',  'ma', 'mightn', 'mustn',  'needn', 'shan', 'shouldn',  'wasn',  'weren',  'wouldn']

df["sw"] = df["RemovePunct"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
df.head()

,reviews,rating,decon,decons,RemovePunct,sw
0,!we enjoy watching this movie with our kids! it's enjoyable for the whole family,5,!we enjoy watching this movie with our kids! it is enjoyable for the whole family,we enjoy watching this movie with our kids it is enjoyable for the whole family,we enjoy watching this movie with our kids it is enjoyable for the whole family,enjoy watching movie kids enjoyable whole family
1,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",3,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",not best sequel but one of better ones it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any...,not best sequel but one of better ones it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any...,not best sequel one better ones builds id4 universe explanation aliens wanted earth also third party race way much chi expected sci fi film day age
2,"""best pressed powder i've ever used. it is so smooth going on my face, and it looks natural.",5,"""best pressed powder I have ever used. it is so smooth going on my face, and it looks natural.",best pressed powder I have ever used it is so smooth going on my face and it looks natural,best pressed powder I have ever used it is so smooth going on my face and it looks natural,best pressed powder i ever used smooth going face looks natural
3,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. doesn't get much better.johnny depp continues a successf...",4,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. does not get much better.johnny depp continues a success...",dark shadows is tim burton s take on the tv show of the same name and a really cool one at that great cast and a cameo of alice cooper does not get much better johnny depp continues a successful p...,dark shadows is tim burton s take on the tv show of the same name and a really cool one at that great cast and a cameo of alice cooper does not get much better johnny depp continues a successful p...,dark shadows tim burton take tv show name really cool one great cast cameo alice cooper not get much better johnny depp continues successful partnership tim burton began ed wood back early 90 chec...
4,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",5,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",easy to assemble lightweight yet seems sturdyhome to sewing supplies portable machine is on bottom shelf just in case so far so goodbegan with two way basics products now own five love the clean l...,easy to assemble lightweight yet seems sturdyhome to sewing supplies portable machine is on bottom shelf just in case so far so goodbegan with two way basics products now own five love the clean l...,easy assemble lightweight yet seems sturdyhome sewing supplies portable machine bottom shelf case far goodbegan two way basics products five love clean look extra storage space


In [15]:
df['RemoveNumb'] = df['sw'].str.replace('\d+', 'number') # to replace no. by ""
df.head()

,reviews,rating,decon,decons,RemovePunct,sw,RemoveNumb
0,!we enjoy watching this movie with our kids! it's enjoyable for the whole family,5,!we enjoy watching this movie with our kids! it is enjoyable for the whole family,we enjoy watching this movie with our kids it is enjoyable for the whole family,we enjoy watching this movie with our kids it is enjoyable for the whole family,enjoy watching movie kids enjoyable whole family,enjoy watching movie kids enjoyable whole family
1,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",3,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",not best sequel but one of better ones it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any...,not best sequel but one of better ones it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any...,not best sequel one better ones builds id4 universe explanation aliens wanted earth also third party race way much chi expected sci fi film day age,not best sequel one better ones builds idnumber universe explanation aliens wanted earth also third party race way much chi expected sci fi film day age
2,"""best pressed powder i've ever used. it is so smooth going on my face, and it looks natural.",5,"""best pressed powder I have ever used. it is so smooth going on my face, and it looks natural.",best pressed powder I have ever used it is so smooth going on my face and it looks natural,best pressed powder I have ever used it is so smooth going on my face and it looks natural,best pressed powder i ever used smooth going face looks natural,best pressed powder i ever used smooth going face looks natural
3,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. doesn't get much better.johnny depp continues a successf...",4,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. does not get much better.johnny depp continues a success...",dark shadows is tim burton s take on the tv show of the same name and a really cool one at that great cast and a cameo of alice cooper does not get much better johnny depp continues a successful p...,dark shadows is tim burton s take on the tv show of the same name and a really cool one at that great cast and a cameo of alice cooper does not get much better johnny depp continues a successful p...,dark shadows tim burton take tv show name really cool one great cast cameo alice cooper not get much better johnny depp continues successful partnership tim burton began ed wood back early 90 chec...,dark shadows tim burton take tv show name really cool one great cast cameo alice cooper not get much better johnny depp continues successful partnership tim burton began ed wood back early number ...
4,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",5,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",easy to assemble lightweight yet seems sturdyhome to sewing supplies portable machine is on bottom shelf just in case so far so goodbegan with two way basics products now own five love the clean l...,easy to assemble lightweight yet seems sturdyhome to sewing supplies portable

In [16]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
  
lemmatizer = WordNetLemmatizer()
  
def lemma(sentence):
  def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
  
  
  
# tokenize the sentence and find the POS tag for each token
  pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
  
  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  
  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
    if tag is None:
        # if there is no available tag, append the token as is
        lemmatized_sentence.append(word)
    else:        
        # else use the tag to lemmatize the token
        lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  lemmatized_sentence = " ".join(lemmatized_sentence)
  
  return lemmatized_sentence

df["text_lemmatized"] = df['RemoveNumb'].apply(lemma)
df.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,reviews,rating,decon,decons,RemovePunct,sw,RemoveNumb,text_lemmatized
0,!we enjoy watching this movie with our kids! it's enjoyable for the whole family,5,!we enjoy watching this movie with our kids! it is enjoyable for the whole family,we enjoy watching this movie with our kids it is enjoyable for the whole family,we enjoy watching this movie with our kids it is enjoyable for the whole family,enjoy watching movie kids enjoyable whole family,enjoy watching movie kids enjoyable whole family,enjoy watch movie kid enjoyable whole family
1,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",3,""" not best sequel but one of better ones"" it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with ...",not best sequel but one of better ones it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any...,not best sequel but one of better ones it builds on id4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any...,not best sequel one better ones builds id4 universe explanation aliens wanted earth also third party race way much chi expected sci fi film day age,not best sequel one better ones builds idnumber universe explanation aliens wanted earth also third party race way much chi expected sci fi film day age,not best sequel one good one build idnumber universe explanation alien want earth also third party race way much chi expect sci fi film day age
2,"""best pressed powder i've ever used. it is so smooth going on my face, and it looks natural.",5,"""best pressed powder I have ever used. it is so smooth going on my face, and it looks natural.",best pressed powder I have ever used it is so smooth going on my face and it looks natural,best pressed powder I have ever used it is so smooth going on my face and it looks natural,best pressed powder i ever used smooth going face looks natural,best pressed powder i ever used smooth going face looks natural,best pressed powder i ever use smooth go face look natural
3,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. doesn't get much better.johnny depp continues a successf...",4,"""dark shadows"" is tim burton's take on the tv show of the same name and a really cool one, at that. great cast and a cameo of alice cooper. does not get much better.johnny depp continues a success...",dark shadows is tim burton s take on the tv show of the same name and a really cool one at that great cast and a cameo of alice cooper does not get much better johnny depp continues a successful p...,dark shadows is tim burton s take on the tv show of the same name and a really cool one at that great cast and a cameo of alice cooper does not get much better johnny depp continues a successful p...,dark shadows tim burton take tv show name really cool one great cast cameo alice cooper not get much better johnny depp continues successful partnership tim burton began ed wood back early 90 chec...,dark shadows tim burton take tv show name really cool one great cast cameo alice cooper not get much better johnny depp continues successful partnership tim burton began ed wood back early number ...,dark shadow tim burton take tv show name really cool one great cast cameo alice cooper not get much well johnny depp continue successful partnership tim burton begin ed wood back early number chec...
4,"""easy to assemble...lightweight....yet seems sturdyhome to sewing supplies, portable machine is on bottom shelf, just in case!so far, so goodbegan with two way basics products, now own five......l...",5,"""easy to assem

In [17]:

# from textblob import TextBlob

# df["spell"] =df["text_lemmatized"].apply(lambda txt: ''.join(TextBlob(txt).correct()))

# df.head()

In [18]:
# Split the dataset into features data and target data

y = df['rating']
X = df['text_lemmatized']



In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=1)

In [20]:
print('X_train :', (len(X_train)))
print('X_test :', (len(X_test)))
print('y_train :', (len(y_train)))
print('y_test :', (len(y_test)))

X_train : 62114
X_test : 15529
y_train : 62114
y_test : 15529


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Tfid_fVectorizer = TfidfVectorizer(lowercase = True)

from sklearn.linear_model import LogisticRegression
Logistic_Regression = LogisticRegression(solver = "lbfgs")
P = Pipeline([('ngram', TfidfVectorizer(ngram_range=(1, 4))),('classifier',Logistic_Regression)])
P.fit(X_train,y_train)

Prediction = P.predict(X_test)

# # P = Pipeline([('vectorizer',Tfid_fVectorizer),('classifier',Logistic_Regression)])
# # P.fit(X_train,y_train)

# # Prediction = P.predict(X_test)




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Prediction, y_test)

array([[ 610,  137,   79,   55,   61],
       [ 126,  309,  138,   10,    5],
       [  58,  122,  358,  156,   48],
       [  34,   73,  341,  939,  443],
       [ 150,  126,  342, 2048, 8761]])

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy_score = accuracy_score(y_test, Prediction)
precision_score = precision_score(y_test, Prediction, average='weighted')
recall_score = recall_score(y_test, Prediction, average='weighted')

print("Model Accuracy on test dataset: ", accuracy_score)
print("Model precision on test dataset: ", precision_score)
print("Model recall on test dataset: ", recall_score)


Model Accuracy on test dataset:  0.7068710155193509
Model precision on test dataset:  0.6718690681601466
Model recall on test dataset:  0.7068710155193509


In [24]:
new_review = ["1st kindle screen failed & had to reboot often. Customer Service couldn't fix. Replaced. 2nd one screen reboots often. Customer service offered replace. Just want one that works. Can't get upgrade replacement","1 dose of Tide HE Turbo Clean in quick cycle vs. 6 doses of the next leading liquid HE compatible detergent in normal cycle.","...well I got it for my girlfriend and she likes it. I guess it was fun to watch",".but I have watched it a few times and it is growing on me.","I bought four of these tablets two of them I need to return because as soon as my nephew's turn them on there were problems with screen being almost completely white and black in the corners you can't see anything I've checked the settings it's not the city it's the tablet SMH I am disappointed","Storks was a film that tried to deliver a simple concept, but it fails due to characters that are unlikable. Some of it is still enjoyable, and has heart value. The jokes really fall flat at times, and it just gives you a silence look at the screen. Pigeon Toady was annoying!","The Secret Life of Pets is a fun and harmless animated Comedy. Nothing more, nothing less.","Enjoyed the sequel a phrase that is not always able to be said. Especially liked the veiled references to the first movie and loved the continuity between the two.","Not best sequel but one of better ones it builds on ID4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any sci fi film this day a age.","Dark Shadows is Tim Burton's take on the TV show of the same name and a really cool one, at that. Great cast and a cameo of Alice Cooper. Doesn't get much better.Johnny Depp continues a successful partnership with Tim Burton which began with Ed Wood back in the early 90's. Check this movie out.","Character development was a major consideration in this latest version, and I actually liked the less-than-realistic/rubber-monster-suit look of the featured characters. It reminds me of the original Godzilla movie I saw as a child. Not a bad movie, but I was expecting more.","Charge the device and was able to set up everything and when I came back to use it it would not work","Change for the better is always appreciated..........in this case,....it failed miserably!!! The new formula along with its new scent misses the mark completely. You have ","Amazon replaced it as it had gray stripes and no picture on the screen. That one works so overall the Fire was a good purchase. Recommend a memory card, it does not hold much","!we enjoy watching this movie with our kids! it's enjoyable for the whole family	","khiya lageko xa khat halka kuina lageko xa scratch xa stting purano vayera buzz sound aairako xa","it's floppy soft n stylish ...best suited for summer n windy climate","Fabric is thinner than I expected. Other than that, no issues. Product is ok overall.","Work well for one two month then it decrease it's performance. So I recommend that this is not good product for used for long term"]
#new_review = ["good","bad","worst","okay","great","love","extreme bad"]
result = P.predict(new_review)
for x in result:
  print(x)


4
5
5
5
1
4
5
5
4
4
4
2
1
4
5
5
5
3
4


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfid_fVectorizer = TfidfVectorizer(lowercase = True)

from sklearn.naive_bayes import MultinomialNB
Multi_nomialNB = MultinomialNB()

In [26]:
from sklearn.pipeline import Pipeline

# P1 = Pipeline([('vectorizer',Tfid_fVectorizer),('classifier', Multi_nomialNB)])
# P1.fit(X_train,y_train)

# Predictions = P1.predict(X_test)

P1 = Pipeline([('ngram', TfidfVectorizer(ngram_range=(1, 3))),('clf',Multi_nomialNB)])
P1.fit(X_train,y_train)

Prediction1 = P1.predict(X_test)

In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Prediction1, y_test)

array([[  21,    1,    0,    0,    0],
       [   0,    0,    0,    0,    0],
       [   0,    0,    1,    0,    0],
       [   5,    7,   35,   43,    0],
       [ 952,  759, 1222, 3165, 9318]])

In [28]:
new_reviews = [".but I have watched it a few times and it is growing on me.","I bought four of these tablets two of them I need to return because as soon as my nephew's turn them on there were problems with screen being almost completely white and black in the corners you can't see anything I've checked the settings it's not the city it's the tablet SMH I am disappointed","Storks was a film that tried to deliver a simple concept, but it fails due to characters that are unlikable. Some of it is still enjoyable, and has heart value. The jokes really fall flat at times, and it just gives you a silence look at the screen. Pigeon Toady was annoying!","The Secret Life of Pets is a fun and harmless animated Comedy. Nothing more, nothing less.","Enjoyed the sequel a phrase that is not always able to be said. Especially liked the veiled references to the first movie and loved the continuity between the two.","Not best sequel but one of better ones it builds on ID4 universe with explanation on what aliens wanted to do with earth and also a third party race but way too much chi which is expected with any sci fi film this day a age.","Dark Shadows is Tim Burton's take on the TV show of the same name and a really cool one, at that. Great cast and a cameo of Alice Cooper. Doesn't get much better.Johnny Depp continues a successful partnership with Tim Burton which began with Ed Wood back in the early 90's. Check this movie out.","Character development was a major consideration in this latest version, and I actually liked the less-than-realistic/rubber-monster-suit look of the featured characters. It reminds me of the original Godzilla movie I saw as a child. Not a bad movie, but I was expecting more.","Charge the device and was able to set up everything and when I came back to use it it would not work","Change for the better is always appreciated..........in this case,....it failed miserably!!! The new formula along with its new scent misses the mark completely. You have ","Amazon replaced it as it had gray stripes and no picture on the screen. That one works so overall the Fire was a good purchase. Recommend a memory card, it does not hold much","!we enjoy watching this movie with our kids! it's enjoyable for the whole family	","khiya lageko xa khat halka kuina lageko xa scratch xa stting purano vayera buzz sound aairako xa","it's floppy soft n stylish ...best suited for summer n windy climate","Fabric is thinner than I expected. Other than that, no issues. Product is ok overall.","Work well for one two month then it decrease it's performance. So I recommend that this is not good product for used for long term"]
result = P1.predict(new_reviews)
for x in result:
  print(x)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy_score = accuracy_score(y_test, Prediction1)
precision_score = precision_score(y_test, Prediction1, average='weighted')
recall_score = recall_score(y_test, Prediction1, average='weighted')

print("Model Accuracy on test dataset: ", accuracy_score)
print("Model precision on test dataset: ", precision_score)
print("Model recall on test dataset: ", recall_score)


Model Accuracy on test dataset:  0.6042243544336403
Model precision on test dataset:  0.6025114336365833
Model recall on test dataset:  0.6042243544336403


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
